In [ ]:
import json
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import pickle
from itertools import combinations

import pprint
import glob
from random import random, randrange
from random import choice
import logging
from collections import defaultdict
import sys

import subprocess
import shlex
from scipy.stats import pearsonr

import pandas as pd

import datetime
from datetime import datetime, timedelta
from dateutil.parser import parse
from collections import Counter
from scipy.signal import savgol_filter
import itertools
import time
import csv

import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


from sklearn.decomposition import LatentDirichletAllocation,NMF

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS


In [ ]:
start_time = time.time()

In [ ]:
# Load network
PT = nx.read_gexf('PT-pruned.gexf')

In [ ]:
clusters = [13,48,6,49,104]#,57,85,80]
cluster_names = ['Democrats',"Maga","Socialists","Health","Antivaxers","French","Indian","FreedomUK"]

In [ ]:
# Delete outside nodes
H = PT.copy()
print(len(H.nodes()))
exclude = []
for node in H.nodes:
    if H.nodes[node]['louvain'] not in clusters:
        exclude.append(node)
        
H.remove_nodes_from(exclude)
print(len(H.nodes())/len(PT.nodes()))

In [ ]:
# out_degree is roughly the amount of people that retweeted them
out_degree = 4
exclude = []
for node in H.nodes:
    if H.out_degree(node)<out_degree:
        exclude.append(node)
            
H.remove_nodes_from(exclude)
print(len(H.nodes())/len(PT.nodes()))

In [ ]:
# edge weight is the amount of times one [person retweeted the other]
edge_weight = 0
exclude = []
for edge in H.edges():
    if H.edges[edge]['weight'] < edge_weight:
        exclude.append(edge)

H.remove_edges_from(exclude)
print(len(H.nodes())/len(PT.nodes()))

In [ ]:
H.remove_nodes_from(list(nx.isolates(H)))
print(len(H.nodes())/len(PT.nodes()))

In [ ]:
# Same amoung per community
equal_pops = False
if equal_pops:
    cluster_size=[]
    for cluster in clusters:
        count = 0
        for node in H.nodes():
            if H.nodes[node]['louvain'] == cluster:
                count += 1
        cluster_size.append(count)
    cut = min(cluster_size)

    def get_bottom(cluster,n):
        ranking = dict()
        for node in H.nodes():
            if H.nodes[node]['louvain'] == cluster:
                ranking[node]=H.nodes[node]['neighbors']
        sort_nodes = sorted(ranking.items(), key=lambda x: x[1], reverse=True)
        sort_nodes = sort_nodes[n:]
        result =[i[0] for i in sort_nodes]
        return result

    exclude = []
    for cluster in clusters:
        ex = get_bottom(cluster,cut)
        exclude.extend(ex)

    #print(exclude)
    H.remove_nodes_from(exclude)
    print(len(H.nodes())/len(PT.nodes()))

In [ ]:
pc = True
if pc:
    giant = max(nx.algorithms.components.weakly_connected_components(H), key=len)
    tocutnodes = []
    for node in H.nodes():
        if node not in giant:
            tocutnodes.append(node)
    H.remove_nodes_from(tocutnodes)
    print(len(H.nodes())/len(PT.nodes()))

In [ ]:
nx.write_gexf(H,'PT-pruned-plotting.gexf')

In [ ]:
elapsed_time = time.time() - start_time
print(elapsed_time)